# Introduction to Computer Vision Assignment2 - Non Local Filtering

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import math
import random
from tqdm import tqdm
import time 
from scipy.spatial import distance_matrix
from scipy import ndimage

## 1. Noise Image generation

In [ ]:
# load data
img_list = []
for i,file_name in enumerate(os.listdir("./data")) :
    image = plt.imread("./data/" + file_name)
    img_list.append(image)

w1,h1,_ = img_list[0].shape
w2,h2,_ = img_list[1].shape

# get gaussian noise
mean = 0
std_list = [20,30,40]
noise_img_list_1 = []
noise_img_list_2 = []

#image with noise 
for std in std_list :   
    noise1 = np.random.normal(mean,std,img_list[0].shape)
    noise2 = np.random.normal(mean,std,img_list[1].shape)
    img1 = np.array(img_list[0], copy=True) 
    img2 = np.array(img_list[1], copy=True)
    cv2.normalize(noise1,noise1,0,255,norm_type=cv2.NORM_MINMAX)
    cv2.normalize(noise2,noise2,0,255,norm_type=cv2.NORM_MINMAX)
    cv2.normalize(img1,img1,0,255,norm_type=cv2.NORM_MINMAX)
    cv2.normalize(img2,img2,0,255,norm_type=cv2.NORM_MINMAX)
    a = np.array(img1 + noise1)
    b = np.array(img2 + noise2)
    cv2.normalize(a,a,0,255,norm_type=cv2.NORM_MINMAX)
    cv2.normalize(b,b,0,255,norm_type=cv2.NORM_MINMAX)
    a = a.astype(np.uint8)
    b = b.astype(np.uint8)
    noise_img_list_1.append(a)
    noise_img_list_2.append(b)

# show original and noise image 
fig = plt.figure(figsize=(10,10))
for i in range(8):
    fig.add_subplot(2, 4, i+1)
    plt.axis("off")
    if i == 0 :
        plt.imshow(img_list[0])
        plt.title("Original Image")
    elif i == 4 :
        plt.title("Original Image")
        plt.imshow(img_list[1])
    elif i > 0 and i < 4 :
        plt.imshow(noise_img_list_1[i-1])
    elif i > 4 : 
        plt.imshow(noise_img_list_2[i-5])
    
    if i % 4 == 1:
        plt.title("noise with σ = {}".format(std_list[0]))
    elif i % 4 == 2:
        plt.title("noise with σ = {}".format(std_list[1]))
    elif i % 4 == 3:
        plt.title("noise with σ = {}".format(std_list[2]))

plt.subplots_adjust(hspace=0, wspace=0)
plt.show()

## 2. Implement Original NLM algorithm

In [ ]:
# Implement the original NLM algorithm with the same parameters as in [1].

def gaussian_kernel(l, sig):
    ax = np.arange(-l // 2 + 1., l // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sig**2))
    final = kernel / kernel.sum()
    return final

def non_local_means_filter(noise_img):
    outputs = np.zeros(noise_img.shape)
    cv2.normalize(noise_img,noise_img,0,1,norm_type=cv2.NORM_MINMAX)
    for t in (range(3)) :
        image = noise_img[:,:,t]
        [m, n] = image.shape
        img = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_DEFAULT)
        out = np.zeros((m, n), dtype='float')
        kernel = gaussian_kernel(7, 1)
        h = 21
        h = h * h
        for i in (range(6, m)):
            for j in range(6,n):
                w1 = img[i:i + 7, j:j + 7]
                wmax = 0
                avg = 0
                sweight = 0
                rmin = i - 2;rmax = i + 2;cmin = j - 2;cmax = j + 2
                for r in range(rmin, rmax):
                    for c in range(cmin, cmax):
                        w2 = img[r - 3:r + 4, c - 3:c + 4]
                        bl = w1 - w2
                        temp = np.multiply(bl, bl)
                        d = sum(sum(np.multiply(kernel, temp)))
                        w = np.exp( -d / h)
                        if w > wmax:
                            wmax = w
                        sweight = sweight + w
                        avg = avg + w * img[r, c]

                avg = avg + wmax * img[i, j]
                sweight = sweight + wmax
                if sweight > 0:
                    out[i-5 , j-5 ] = avg / sweight
                else:
                    out[i-5 , j-5 ] = img[i, j]
        outputs[:,:,t] = out
    cv2.normalize(outputs,outputs,0,255,norm_type=cv2.NORM_MINMAX)
    outputs = outputs.astype(np.uint8)
    return outputs


start_time = time.time()
fig = plt.figure(figsize=(10,10))
nlmf_img = []
for i in tqdm(range(8)):
    fig.add_subplot(2, 4, i+1)
    plt.axis("off")
    temp = np.zeros(img_list[0].shape)
    if i == 0 :
        temp = img_list[0]
    elif i == 4 :
        temp = img_list[1]
    elif i > 0 and i < 4 :
        temp = non_local_means_filter(noise_img_list_1[i-1])
    elif i > 4 : 
        temp = non_local_means_filter(noise_img_list_2[i-5])
    plt.imshow(temp)
    nlmf_img.append(temp)
    if i % 4 == 1:
        plt.title("denoising with σ = {}".format(std_list[0]))
    elif i % 4 == 2:
        plt.title("denoising with σ = {}".format(std_list[1]))
    elif i % 4 == 3:
        plt.title("denoising with σ = {}".format(std_list[2]))
    elif i % 4 == 0 :
        plt.title("Original Image")
end_time = time.time()
print("Time : {} sec".format(str(int(end_time-start_time))))
plt.subplots_adjust(hspace=0, wspace=0)
plt.show()

## 3. Rotation and Mirror Matching

In [ ]:
# In NLM, finding many similar patches are crucial to the denoising quality. In the original
# version [1], it only uses the candidate patches of the same orientation. However, if we consider
# rotation and mirror symmetry of a patch, much more similar candidate patches can be obtained.
# So, modify your algorithm so that you can utilize rotated and mirrored similar patches in the
# search window.
# For rotation-invariant patch matching, you may need to find the orientations of two patches and
# normalize the orientations of them before matching. To determine the dominant orientation of a
# patch, you can use the dominant gradient orientation as used in SIFT descriptor.

def gaussian_kernel(l, sig):
    ax = np.arange(-l // 2 + 1., l // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sig**2))
    final = kernel / kernel.sum()
    return final

def dominant_gradient_orientation_nlm_filter(noise_img):
    outputs = np.zeros(noise_img.shape)
    np.seterr(divide='ignore', invalid='ignore')
    cv2.normalize(noise_img,noise_img,0,1,norm_type=cv2.NORM_MINMAX)
    for t in (range(3)) :
        image = noise_img[:,:,t]
        [m, n] = image.shape
        img = cv2.copyMakeBorder(image, 7, 7, 7, 7, cv2.BORDER_DEFAULT)
        out = np.zeros((m, n), dtype='float')
        kernel = gaussian_kernel(7, 1)
        h = 21
        h = h * h
        for i in (range(8, m)):
            for j in range(8,n):
                w1 = img[i:i + 7, j:j + 7]
                # get dominant gradient orientation
                gradient_w1_x = cv2.Sobel(w1,cv2.CV_64F,1,0,3)
                gradient_w1_y = cv2.Sobel(w1,cv2.CV_64F,0,1,3)    
                mag_w1 = np.sqrt(gradient_w1_x*gradient_w1_x+gradient_w1_y*gradient_w1_y)
                angle_w1 = np.arctan2(gradient_w1_y,gradient_w1_x)
                o_w1 = np.multiply(mag_w1,gaussian_kernel(7,1.5*3))
                o_w1 -= np.max(angle_w1)
                # 
                wmax = 0
                avg = 0
                sweight = 0
                rmin = i - 2;rmax = i + 2;cmin = j - 2;cmax = j + 2
                for r in range(rmin, rmax):
                    for c in range(cmin, cmax):
                        # get dominant gradient orientation
                        w2 = img[r - 3:r + 4, c - 3:c + 4]
                        gradient_w2_x = cv2.Sobel(w2,cv2.CV_64F,1,0,3)
                        gradient_w2_y = cv2.Sobel(w2,cv2.CV_64F,0,1,3)
                        mag_w2 = np.sqrt(gradient_w2_x*gradient_w2_x+gradient_w2_y*gradient_w2_y)
                        angle_w2 = np.arctan2(gradient_w2_y,gradient_w2_x)
                        o_w2 = np.multiply(mag_w2,gaussian_kernel(7,1.5*3))
                        o_w2 -= np.max(angle_w2)
                        #
                        mm = np.subtract(o_w1,o_w2)
                        bl = np.linalg.norm(mm)
                        temp = np.multiply(bl, bl)
                        d = sum(sum(np.multiply(kernel, temp)))
                        w = np.exp(-d / h)
                        if w > wmax:
                            wmax = w
                        sweight = sweight + w
                        avg = avg + w * img[r, c]

                avg = avg + wmax * img[i, j]
                sweight = sweight + wmax
                if sweight > 0:
                    out[i-7 , j-7 ] = avg / sweight
                else:
                    out[i-7 , j-7 ] = img[i, j]
        outputs[:,:,t] = out
    w,h,_ = outputs.shape
    cv2.normalize(outputs,outputs,0,255,norm_type=cv2.NORM_MINMAX)
    outputs = outputs.astype(np.uint8)
    return outputs

start_time = time.time()
fig = plt.figure(figsize=(10,10))
dgo_nlmf_img = []
for i in tqdm(range(8)):
    fig.add_subplot(2, 4, i+1)
    plt.axis("off")
    temp = np.zeros(img_list[0].shape)
    if i == 0 :
        temp = img_list[0]
    elif i == 4 :
        temp = img_list[1]
    elif i > 0 and i < 4 :
        temp = dominant_gradient_orientation_nlm_filter(noise_img_list_1[i-1])
    elif i > 4 : 
        temp = dominant_gradient_orientation_nlm_filter(noise_img_list_2[i-5])
    plt.imshow(temp)
    dgo_nlmf_img.append(temp)
    if i % 4 == 1:
        plt.title("denoising with σ = {}".format(std_list[0]))
    elif i % 4 == 2:
        plt.title("denoising with σ = {}".format(std_list[1]))
    elif i % 4 == 3:
        plt.title("denoising with σ = {}".format(std_list[2]))
    elif i % 4 == 0 :
        plt.title("Original Image")
end_time = time.time()
print("Time : {} sec".format(str(int(end_time-start_time))))
plt.subplots_adjust(hspace=0, wspace=0)
plt.show()

## 4. Scale-space search

In [ ]:
# construct the Gaussian pyramid of the noisy image in three levels
noise_img_list = [noise_img_list_1,noise_img_list_2]
t = 1
fig = plt.figure(figsize=(30,10))
gaussian_pyramid_list = [] 
for noise_img in noise_img_list :
    for idx in range(len(noise_img)) : 
        layer = noise_img[idx].copy()
        gaussian_pyramid = [layer]
        h,w,d = layer.shape
        for i in range(2):
            layer = cv2.pyrDown(layer)
            gaussian_pyramid.append(layer)
            w += layer.shape[1]
        gaussian_pyramid_list.append(gaussian_pyramid)
        pyr1 = np.ones((h,w,d))
        w,w2,h,h2 = 0,0,0,0

        for i in range(len(gaussian_pyramid)) :
            layer = gaussian_pyramid[i]
            w2 += layer.shape[1]
            pyr1[h:,w:w2,:] = layer
            h += layer.shape[0]//2
            w += layer.shape[1]
        fig.add_subplot(2, 3, t)
        if t % 3 == 1 :
            plt.title("Noise with with σ = 20")
        elif t % 3 == 2 :
            plt.title("Noise with with σ = 30")
        elif t % 3 == 0 :
            plt.title("Noise with with σ = 40")
        t+=1
        cv2.normalize(pyr1,pyr1,0,255,norm_type=cv2.NORM_MINMAX)
        pyr1 = pyr1.astype(np.uint8)
        plt.imshow(pyr1)
        plt.axis("off")
        
plt.subplots_adjust(hspace=0.1, wspace=0)
plt.show()

In [ ]:
# Try to find the similar patches not only in the same scale but also in other scaled images. You
# can keep the size of the search window the same in all scaled images.
# Implement the original NLM algorithm with the same parameters as in [1].
def gaussian_kernel(l, sig):
    ax = np.arange(-l // 2 + 1., l // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sig**2))
    final = kernel / kernel.sum()
    return final

def scale_local_means_filter(noise_img,gps):
    outputs = np.zeros(noise_img.shape)
    cv2.normalize(noise_img,noise_img,0,1,norm_type=cv2.NORM_MINMAX)
    # padding
    padding_gp = []
    for gp in reversed(gps) :         
        cv2.normalize(gp,gp,0,1,norm_type=cv2.NORM_MINMAX)
        padding_gp.append(cv2.copyMakeBorder(gp, 5, 5, 5, 5, cv2.BORDER_DEFAULT))
    for t in (range(3)) :
        image = noise_img[:,:,t]
        [m, n] = image.shape
        img = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_DEFAULT)
        out = np.zeros((m, n), dtype='float')
        kernel = gaussian_kernel(7, 1)
        h = 21
        h = h * h
        for i in (range(6, m)):
            for j in range(6,n):
                w1 = img[i:i + 7, j:j + 7]
                wmax = 0
                avg = 0
                sweight = 0
                rmin = i - 2;rmax = i + 2;cmin = j - 2;cmax = j + 2
                for gp in padding_gp :
                    if rmax + 4 > gp.shape[0] or cmax + 4 > gp.shape[1] :
                        continue
                    for r in range(rmin, rmax):
                            for c in range(cmin, cmax):
                                w2 = gp[r - 3:r + 4, c - 3:c + 4,t]
                                bl = w1 - w2
                                d = np.multiply(bl, bl)
                                d = np.multiply(kernel,d)
                                d = np.linalg.norm(d)
                                w = np.exp(d / h)
                                if w > wmax:
                                    wmax = w
                                sweight = sweight + w
                                avg = avg + w * img[r, c]

                avg = avg + wmax * img[i, j]
                sweight = sweight + wmax
                if sweight > 0:
                    out[i-5 , j-5 ] = avg / sweight
                else:
                    out[i-5 , j-5 ] = img[i, j]
        outputs[:,:,t] = out
    cv2.normalize(outputs,outputs,0,255,norm_type=cv2.NORM_MINMAX)
    outputs = outputs.astype(np.uint8)
    return outputs

start_time = time.time()
fig = plt.figure(figsize=(10,10))
scale_nlmf_img = []
for i in tqdm(range(8)):
    fig.add_subplot(2, 4, i+1)
    plt.axis("off")
    temp = np.zeros(img_list[0].shape)
    if i == 0 :
        temp = img_list[0]
    elif i == 4 :
        temp = img_list[1]
    elif i > 0 and i < 4 :
        temp = scale_local_means_filter(noise_img_list_1[i-1],gaussian_pyramid_list[i-1])
    elif i > 4 : 
        temp = scale_local_means_filter(noise_img_list_2[i-5],gaussian_pyramid_list[i-2])
    plt.imshow(temp)
    scale_nlmf_img.append(temp)
    if i % 4 == 1:
        plt.title("denoising with σ = {}".format(std_list[0]))
    elif i % 4 == 2:
        plt.title("denoising with σ = {}".format(std_list[1]))
    elif i % 4 == 3:
        plt.title("denoising with σ = {}".format(std_list[2]))
    elif i % 4 == 0 :
        plt.title("Original Image")
end_time = time.time()
print("Time : {} sec".format(str(int(end_time-start_time))))
plt.subplots_adjust(hspace=0, wspace=0)
plt.show()

## PSNR
#### Report the quality of the filtered images using PSNR and the running time for each algorithm you have implemented.

In [ ]:
def get_PSNR(original, noise_img, maxi=255):
    mse = np.mean((original-noise_img)**2)
    if mse == 0 :
        return 100
    PSNR = 20*math.log10(maxi/math.sqrt(mse))
    
    return PSNR
#
img1 = np.array(img_list[0], copy=True) 
img2 = np.array(img_list[1], copy=True)
cv2.normalize(img1,img1,0,255,norm_type=cv2.NORM_MINMAX)
cv2.normalize(img2,img2,0,255,norm_type=cv2.NORM_MINMAX)

print("--------------------------------------------------------------PSNR SCORE--------------------------------------------------------------")
for i in range(1,len(nlmf_img)) :
    psnr4=
    if i < 4 : 
        psnr1 = get_PSNR(img1,noise_img_list_1[i-1])
        psnr2 = get_PSNR(img1,nlmf_img[i])
        psnr3 = get_PSNR(img1,dgo_nlmf_img[i])
        psnr4 = get_PSNR(img1,scale_nlmf_img[i])
        print("Noise image1 with σ = {} : {:6.5} | NLM filter : {:6.5} | Dominant Orientation gradient NLM filter : {:.5} | Scale Space Search : {:.5}".format(std_list[i-1],psnr1,psnr2,psnr3,psnr4))
    elif i == 4:
        continue
    else :
        psnr1 = get_PSNR(img2,noise_img_list_2[i-5])
        psnr2 = get_PSNR(img2,nlmf_img[i])
        psnr3 = get_PSNR(img2,dgo_nlmf_img[i])
        psnr4 = get_PSNR(img2,scale_nlmf_img[i])
        print("Noise image2 with σ = {} : {:6.5} | NLM filter : {:6.5} | Dominant Orientation gradient NLM filter : {:.5} | Scale Space Search : {:.5}".format(std_list[i-5],psnr1,psnr2,psnr3,psnr4))
